In [1]:
#replace Pubchemid with Smiles and merge known Smiles list with imputed Smiles
!python putSmiles4PubChemID.py data/input/pubchem2smiles.txt data/input/Drug_info_release.csv 


In [2]:
# create fingerprint features
!python createFingerprintFeatures.py drug_smiles.csv > data/features/drugs-fingerprint-matrix.txt


In [3]:
#  create target feature (binary matrix)  If drugsin combination that have a target in Target Set, its corresponding entry is  1, otherwise 0
!python createTargetFeatureMatrix.py data/input/Drug_info_release.csv data/input/targets-expansion.txt >  data/features/drugs-commontarget-matrix.txt


In [5]:
# create common pathway feature (binary matrix)  if drugs target share common pathway  1, otherwise 0
!python createCommonPathwayFeatureMatrix.py data/input/Drug_info_release.csv data/input/targets-expansion.txt data/input/kegg-cancerpathway-mapping.txt > data/features/drugs-commonpathway-matrix.txt 


In [12]:
# create protein domain features
!python createProteinDomainFeatures.py data/input/Drug_info_release.csv data/input/targets-expansion.txt data/input/uniprot-hugo-mapping.txt data/input/domainAnalysis_Pfam.csv data/input/domainAnalysis_SMART.csv data/input/domainAnalysis_ProSiteProfiles.csv data/input/domainAnalysis_ProSitePatterns.csv data/input/domainAnalysis_SUPERFAMILY.csv > data/features/drugs-domains.txt


In [7]:
# calculate drug-drug pair topological features using synergistic drug network
!python buildSynergticDrugCombNetwork.py 

117
DDI size:  67
non-DDI size:  6719
Train size:  6786
2019-04-01 19:55:10 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
<SparkContext master=local[10] appName=pyspark-shell>
2019-04-01 19:55:13 WARN  TaskSetManager:66 - Stage 0 contains a task of very large size (139 KB). The maximum recommended task size is 100 KB.
Output file was stored in data/features/drug-synergy-network-features.txt


In [45]:
# mean gene expression value for each clustered module  
!python createMeanExprFeatureForGeneModule.py data/input/gex.csv data/input/GeneModules  > data/features/cell-gex-module-mean.txt


In [ ]:
# mutations filtered by genes in cancer pathway and its expression varying significanty 
!python filterMutationsByCancerPathway.py ../data/input-data/gex.csv ../data/input-data/mutations.csv ../data/input-data/kegg-cancerpathway-mapping.txt > ../data/features/Dream10-cell-mutations-cancerpathway-significant.txt 



In [ ]:
# cnv filtered by genes correlated its gene expression with copy numbers significantly (above the mean correlation) and  involved in the KEGG cancer pathway
!python createCNVFeaturesByGeneLevel.py ../data/input-data/gex.csv ../data/input-data/cnv_gene.csv ../data/input-data/kegg-cancerpathway-mapping.txt  > ../data/features/Dream10-cell-cnv-genes-cancerpathway.txt 

# cell clinical features
!python findCoverageOfTermsInOnto.py ../data/input-data/cellosaurus.txt ../data/input-data/cell_info.csv 

In [16]:
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")

In [13]:
import csv
import numpy as np
import sys
import pandas as pd
import itertools
import math
import time

from sklearn import svm, linear_model, neighbors
from sklearn import tree, ensemble

from sklearn.naive_bayes import GaussianNB

import networkx as nx
import random
import numbers

In [14]:
from utils import *

In [15]:
def crossvalid(train_df, test_df, model): 
    features_cols= train_df.columns.difference(['COMPOUND_A','COMPOUND_B' ,'SYNERGY_SCORE', 'CELL_LINE','COMBINATION_ID'])
    X=train_df[features_cols].values
    y=train_df['SYNERGY_SCORE'].values.ravel()

    X_new=test_df[features_cols].values
    y_new=test_df['SYNERGY_SCORE'].values.ravel()

    model.fit(X,y)
    probs= model.predict(X_new)
    
    pred_df = test_df[['SYNERGY_SCORE', 'CELL_LINE','COMBINATION_ID']]
    pred_df['PREDICTION'] = probs
    model_scores = get_scores(model, X_new, y_new)
    model_scores['primaryMetric']=primaryMetric(pred_df)
    #lr_scores = get_scores(lr, X_new, y_new)
    
    return model_scores#, sclf_scores


In [17]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext

In [18]:
if False: 
    sc.stop()

config = SparkConf()
config.setMaster("local[10]")
config.set("spark.executor.memory", "70g")
config.set('spark.driver.memory', '90g')
config.set("spark.memory.offHeap.enabled",True)
config.set("spark.memory.offHeap.size","50g") 
sc = SparkContext(conf=config)
print (sc)

<SparkContext master=local[10] appName=pyspark-shell>


In [21]:
drugfeatfiles = ['drugs-commonpathway-matrix.txt', 'drugs-commontarget-matrix.txt', 'drugs-domains.txt','drugs-fingerprint-matrix.txt', 'drug-synergy-network-features.txt'] # 'network-sim.txt' 'Dream10-drugs-topo-sim.txt'
diseasefeatfiles = ['Dream10-cell-cnv-genes-cancerpathway.txt','Dream10-cell-gex-module-mean.txt','Dream10-cell_phenotype_features.txt', 'Dream10-cell-mutations-cancerpathway-significant.txt' ]


In [22]:
folder ='data/features/'
for i,featureFilename in enumerate(drugfeatfiles):
    temp=pd.read_csv(folder+featureFilename, delimiter='\t')
    if i != 0:
        drug_df=drug_df.merge(temp,on=['Drug1','Drug2'],how='outer')
        #drug_df=drug_df.merge(temp,how='outer',on='Drug')
    else:
        drug_df =temp
    print (featureFilename,' # columns :',len(temp.columns))
drug_df.fillna(0, inplace=True)

drugs-commonpathway-matrix.txt  # columns : 142
drugs-commontarget-matrix.txt  # columns : 187
drugs-domains.txt  # columns : 372
drugs-fingerprint-matrix.txt  # columns : 258
drug-synergy-network-features.txt  # columns : 5


In [24]:
len(drug_df)

14161

In [25]:
folder ='data/features/'
for i,featureFilename in enumerate(diseasefeatfiles):
    temp=pd.read_csv(folder+featureFilename, delimiter='\t')
    if i != 0:
        cell_df = cell_df.merge(temp,on=['CellLine'], how='outer')
        #drug_df=drug_df.merge(temp,how='outer',on='Drug')
    else:
        cell_df = temp
    print (featureFilename,' # columns :',len(temp.columns))
cell_df.fillna(0, inplace=True)

Dream10-cell-cnv-genes-cancerpathway.txt  # columns : 144
Dream10-cell-gex-module-mean.txt  # columns : 54
Dream10-cell_phenotype_features.txt  # columns : 5
Dream10-cell-mutations-cancerpathway-significant.txt  # columns : 877


In [26]:
len(cell_df)

85

In [29]:
combo_df = pd.read_csv('data/input/ch1_train_combination_and_monoTherapy.csv', delimiter=',')
len(combo_df)

2199

In [31]:
combo_df1 = pd.read_csv('data/input/ch1_LB.csv', delimiter=',')

combo_df = combo_df.append(combo_df1)
print (len(combo_df1),len(combo_df))

591 2790


In [17]:
#combo_df2 = pd.read_csv('data/input-data/ch2_LB.csv', delimiter=',')

#combo_df = combo_df.append(combo_df2)
#print (len(combo_df2),len(combo_df))

In [18]:
#len(combo_df.groupby(['CELL_LINE','COMBINATION_ID']))

In [33]:
cell_df.rename(columns={'CellLine':'CELL_LINE'}, inplace=True)
drug_df.rename(columns={'Drug1':'COMPOUND_A', 'Drug2':'COMPOUND_B'}, inplace=True)

In [34]:
combo_df = combo_df.merge(drug_df, on=['COMPOUND_A','COMPOUND_B']).merge(cell_df, on='CELL_LINE')
len(combo_df.columns)

2044

In [35]:
len(combo_df)

2790

In [36]:
def cv_run(model, combo_df, drug_df, cell_df, train, test):
    print( len(train),len(test))
    train_df = combo_df.iloc[train]

    test_df = combo_df.iloc[test]
    model_scores= crossvalid(train_df, test_df, model)
    return model_scores

def cvSpark(model, combo_df, drug_df, cell_df, cv):    
    #print (cv)
    rdd = sc.parallelize(cv).map(lambda x: cv_run(model, combo_df, drug_df, cell_df, x[0], x[1] ))
    all_scores = rdd.collect()
    return pd.DataFrame(all_scores)

In [39]:
from sklearn import cross_validation 
import xgboost as xgb
from sklearn import svm
from sklearn import metrics

In [ ]:
# all_scores_df = pd.DataFrame()
n_seed =100

combo_df_bc= sc.broadcast(combo_df)
drug_df_bc= sc.broadcast(drug_df)
cell_df_bc= sc.broadcast(cell_df)
n_run = 1
n_fold = 10
for i in range(n_run):
    n_seed +=1
    print ('run',i)
    cv = cross_validation.KFold(len(combo_df), n_folds=10, shuffle=True, random_state=n_seed)
    #cv = cross_validation.StratifiedKFold(y=tuples, n_folds=n_fold, shuffle=True, random_state=n_seed)
    cv_list = [ (train,test) for i, (train, test) in enumerate(cv)]


    rf_model = ensemble.RandomForestRegressor(n_estimators=200, n_jobs=10)
    rf_scores =cvSpark(rf_model, combo_df_bc.value, drug_df_bc.value, cell_df_bc.value, cv_list)
    lr_model = linear_model.LinearRegression()
    lr_scores =cvSpark(lr_model, combo_df_bc.value, drug_df_bc.value, cell_df_bc.value, cv_list)
    gbm = xgb.XGBRegressor(max_depth=10, n_estimators=250, learning_rate=0.05, 
                           colsample_bylevel= 0.8, subsample= 0.75)
    gbm_scores =cvSpark(gbm, combo_df_bc.value, drug_df_bc.value, cell_df_bc.value, cv_list)
    lasso = linear_model.Lasso(alpha=0.01)
    lasso_scores =cvSpark(lasso, combo_df_bc.value, drug_df_bc.value, cell_df_bc.value, cv_list)
    svm_model = svm.LinearSVR()
    svm_scores =cvSpark(svm_model, combo_df_bc.value, drug_df_bc.value, cell_df_bc.value, cv_list)
    lr_scores = lr_scores.mean()
    gbm_scores = gbm_scores.mean()
    rf_scores = rf_scores.mean()
    lasso_scores = lasso_scores.mean()
    svm_scores = svm_scores.mean()
    
    lr_scores['method']= 'LinReg'
    gbm_scores['method']= 'XGBoost'
    rf_scores['method']= 'Random Forest'
    lasso_scores['method']= 'Lasso'
    svm_scores['method']= 'SVM'

    all_scores_df = all_scores_df.append(lr_scores.mean(), ignore_index=True)
    all_scores_df = all_scores_df.append(gbm_scores, ignore_index=True)
    all_scores_df = all_scores_df.append(rf_scores, ignore_index=True)
    all_scores_df = all_scores_df.append(lasso_scores, ignore_index=True)
    all_scores_df = all_scores_df.append(svm_scores, ignore_index=True)

In [34]:
all_scores_df.groupby('method').mean()

,neg_mean_squared_error,primaryMetric,r2
method,,,
Lasso,-2.223359e+04,0.241981,-1.921940e+00
LinReg,-1.714133e+21,0.241415,-7.134217e+17
Random Forest,-2.033077e+04,0.379679,-1.601775e+00
SVM,-1.851414e+04,0.313326,1.859608e-01
XGBoost,-1.884207e+04,0.393769,-9.954959e-01


In [189]:
all_scores_df.to_csv('data/results/modelComparison_ch1_train_lb_Abridge_10of10-CV.csv',sep=',')

## Tuning the Xgboost parameters

In [26]:

def cvSparkXGboost(train_df, test_df, params):    
    #print (cv)
    rdd = sc.parallelize(params).map(lambda param: cv_run_gbm( train_df, test_df, param))
    all_scores = rdd.collect()
    return pd.DataFrame(all_scores)


def fold_gbm(combo_df, drug_df, cell_df, param):
    gbm = xgb.XGBRegressor(max_depth=param['max_depth'], n_estimators=param['n_estimators'], learning_rate=0.05, 
                           subsample=param['subsample'], colsample_bytree=param['colsample_bytree']) 
    #gbm = xgb.XGBRegressor(max_depth=12, n_estimators=100, learning_rate=0.05,  colsample_bylevel= 0.8, colsample_bytree= 0.8)

    #print( len(train),len(test))
    train = param['train']
    test = param['test']
    train_df = combo_df.iloc[train]

    test_df = combo_df.iloc[test]
    
    train_df = train_df.merge(drug_df, on=['COMPOUND_A','COMPOUND_B']).merge(cell_df, on='CELL_LINE')

    test_df = test_df.merge(drug_df, on=['COMPOUND_A','COMPOUND_B']).merge(cell_df, on='CELL_LINE')
    
    gbm_scores= crossvalid(train_df, test_df, gbm)
    gbm_scores['seed'] = param['seed']
    gbm_scores['run'] = param['run']
    gbm_scores['fold'] = param['fold']
    gbm_scores['max_depth']= param['max_depth']
    gbm_scores['n_estimators']= param['n_estimators']
    gbm_scores['subsample']= param['subsample']
    gbm_scores['colsample_bytree']= param['colsample_bytree']
    return gbm_scores

def cvSparkXGboost(combo_df, drug_df, cell_df, params):    
    #print (cv)
    rdd = sc.parallelize(params).map(lambda param: fold_gbm(combo_df, drug_df, cell_df, param))
    all_scores = rdd.collect()
    return pd.DataFrame(all_scores)

In [ ]:
#subsample = c(0.5, 0.75, 1), 
#colsample_bytree = c(0.6, 0.8, 1),
#maxTreeDepth = c(8, 10, 12, 15))

params=[]
n_seed = 100
for run in range(10):
    n_seed +=1
    print ('run',)
    #cv = cross_validation.KFold(len(combo_df), n_folds=10, shuffle=True, random_state=n_seed)
    cv = cross_validation.StratifiedKFold(y=tuples, n_folds=10, shuffle=True, random_state=n_seed)
    for fold, (train, test) in enumerate(cv):
        for subsample in [0.75, 0.9, 1]:
            for colsample_bytree in [0.8, 1]:
                for n_tree in [100, 250, 500]:
                    param={'max_depth':8,'n_estimators':n_tree,'subsample':subsample,'colsample_bytree':colsample_bytree}
                    print (param)
                    param.update({'seed':n_seed,'run':run,'fold':fold,'train':train,'test':test})
                    params.append(param)




In [ ]:
combo_df_bc= sc.broadcast(combo_df)
drug_df_bc= sc.broadcast(drug_df)
cell_df_bc= sc.broadcast(cell_df)
params_bc= sc.broadcast(params)
gbm_scores =cvSparkXGboost(combo_df_bc.value, drug_df_bc.value, cell_df_bc.value, params_bc.value)

In [32]:
for g, df in gbm_scores.groupby(['subsample','colsample_bytree','n_estimators','max_depth']):
    scores=[]
    for r, dfr in df.groupby('run'):
        scores.append(dfr['primaryMetric'].mean())
    print (g, np.mean(scores))

(0.75, 0.8, 100, 8) 0.3687445065183805
(0.75, 0.8, 250, 8) 0.37762284826484727
(0.75, 0.8, 500, 8) 0.3821622761908202
(0.75, 1.0, 100, 8) 0.3694247279392223
(0.75, 1.0, 250, 8) 0.3835034952694392
(0.75, 1.0, 500, 8) 0.38964455294217715
(0.9, 0.8, 100, 8) 0.364997778457321
(0.9, 0.8, 250, 8) 0.3783834937695888
(0.9, 0.8, 500, 8) 0.3791122789325035
(0.9, 1.0, 100, 8) 0.36004792468089974
(0.9, 1.0, 250, 8) 0.3731035874069096
(0.9, 1.0, 500, 8) 0.3739782441183055
(1.0, 0.8, 100, 8) 0.35967130728078445
(1.0, 0.8, 250, 8) 0.37353486761702326
(1.0, 0.8, 500, 8) 0.3720529847211161
(1.0, 1.0, 100, 8) 0.35424626175076857
(1.0, 1.0, 250, 8) 0.3612931476505155
(1.0, 1.0, 500, 8) 0.3651421662615035


In [196]:
for g, df in gbm_scores.groupby(['subsample','colsample_bytree','n_estimators','max_depth']):
    scores=[]
    for r, dfr in df.groupby('run'):
        scores.append(dfr['primaryMetric'].mean())
    print (g, np.mean(scores))

(0.75, 0.8, 250, 8) 0.37762284826484727
(0.75, 0.8, 250, 10) 0.3749030091585308
(0.75, 0.8, 250, 12) 0.38115003324146757
(0.75, 0.8, 250, 15) 0.3747319481988999
(0.75, 1.0, 250, 8) 0.3835034952694392
(0.75, 1.0, 250, 10) 0.3797580812983242
(0.75, 1.0, 250, 12) 0.37605324740554724
(0.75, 1.0, 250, 15) 0.38253286306722256
(0.9, 0.8, 250, 8) 0.3783834937695888
(0.9, 0.8, 250, 10) 0.37706728957598923
(0.9, 0.8, 250, 12) 0.3742196582115166
(0.9, 0.8, 250, 15) 0.3698577660573946
(0.9, 1.0, 250, 8) 0.3731035874069096
(0.9, 1.0, 250, 10) 0.3682319399302032
(0.9, 1.0, 250, 12) 0.36665461249304054
(0.9, 1.0, 250, 15) 0.3635715406352677
(1.0, 0.8, 250, 8) 0.37353486761702326
(1.0, 0.8, 250, 10) 0.3704579192594775
(1.0, 0.8, 250, 12) 0.3687655362262067
(1.0, 0.8, 250, 15) 0.3646992652383646
(1.0, 1.0, 250, 8) 0.3612931476505155
(1.0, 1.0, 250, 10) 0.3648582734354161
(1.0, 1.0, 250, 12) 0.3566880419221299
(1.0, 1.0, 250, 15) 0.3445527616536744


In [34]:
all_df = pd.DataFrame()
for g1, df in gbm_scores.groupby(['subsample','colsample_bytree','n_estimators','max_depth']):
    for g2, x_df in df.groupby('run'): 
        #print (g1,g2)
        all_df= all_df.append(x_df.mean(), ignore_index=True)

In [35]:
all_df.to_csv('data/results/xgboost_gridsearch_ch1_train_lb_Abridge_10of10-CV.csv',sep=',')